# Урок 7. Нестандартные ситуации

In [1]:
import numpy as np
#import pandas as pd

from scipy import stats

import warnings
warnings.filterwarnings('ignore')

## Задание 1
Мы провели A/A текст, в котором сравнили одну и ту же версию сайта. Полученные результаты выглядят следующим образом. Можем ли мы на основе проведенного A/A теста, запустить процесс A/B-тестирования? Используйте уровень доверия 95%.

Для оценки статистической значимости различий между конверсиями в двух группах будем использовать Z-критерий для разности долей:\
$Z = \cfrac{p_1 - p_2}{\sqrt{p (1-p) \left(\cfrac{1}{n_1} + \cfrac{1}{n_2}\right)}}$, где
$p = \cfrac{m_1 + m_2}{n_1 + n_2}$

На уровне значимости $\alpha = 0.05$ проверим гипотезу $H_0: p_1 = p_2$ о том, что выборки равноценны против конкурирующей гипотезы $H_1: p_1 < p_2$.

In [2]:
n1, n2, m1, m2, alpha = 1731, 1702, 152, 189, 0.05

In [3]:
p1, p2, p = m1/n1, m2/n2, (m1+m2)/(n1+n2)

Zk = stats.norm.ppf(1-alpha)
print(f'{Zk = :.2f}')

Zn = (p1-p2)/np.sqrt(p*(1-p)*(1/n1+1/n2))
print(f'{Zn = :.2f}')

print(f'H0: p1 = p2 is {Zn>-Zk}')

Zk = 1.64
Zn = -2.28
H0: p1 = p2 is False


На уровне значимости $\alpha=0.05$ гипотезу $H_0: p_1 = p_2$ отвергаем в пользу конкурирующей гипотезы $H_1: p_1 < p_2$.\

## Задание 2
Мы провели A/A текст, в котором сравнили одну и ту же версию сайта. Полученные результаты выглядят следующим образом. Можем ли мы на основе проведенного A/A теста, запустить процесс A/B-тестирования? Используйте уровень доверия 95%.

$H_0: \bar{X} - \bar{Y} = 0$ \
$H_1: \bar{X} - \bar{Y} > 0$ – правосторонняя критическая область

Для проверки нулевой гипотезы используют статистический критерий \
$Z = \cfrac{\bar{x}-\bar{y}}{\sqrt{\cfrac{D_x^2}{n}+\cfrac{D_y^2}{m}}}$, где \
$\bar{x}$, $\bar{y}$ - выборочные средние \
$D_x^2$, $D_y^2$ - смещенные выборочные дисперсии (можно использовать и несмещенную)

Правосторонняя критическая область. Если $z_н < z_к$, то гипотеза $H_0$ принимается, в случае $z_н > z_к$ – отвергается.

In [4]:
Mx, SDx, Nx = 38.175, 28, 108 #A
My, SDy, Ny = 33.710, 11, 105 #A1
alpha = 0.05

In [5]:
Zk = stats.norm.ppf(1-alpha)
print(f'Zk = {Zk:.2f}')

Zn = (Mx-My)/np.sqrt(SDx**2/Nx + SDy**2/Ny)
print(f'Zn = {Zn:.2f}')

print(f'H0: M(X)=M(Y) is {Zn<Zk}')

Zk = 1.64
Zn = 1.54
H0: M(X)=M(Y) is True


## Задание 3
Мы запустили A/B-тест. На его проведение мы собираемся потратить 1 неделю, на каждую выборку направить по 10000 юзеров. На третий день, продакт посмотрел на промежуточные результаты. На этот момент в каждой выборке было по 3000 юзеров, а результат в конверсиях был статистически значимым в пользу версии B. Можем ли мы на основе таких данных выпустить версию B на всех юзеров?

Нет.

## Задание 4
Мы провели A/B-тест, в котором изменили элементы после добавления товара в корзину. То есть, юзеры видят разные версии сайта после того, как что-то добавят в корзину. Получили следующие результаты. Какую версию вы в итоге выберите?

|  | A | B |
|---|---|---|
| Посещение | 6351 | 6442 |
| Добавление в корзину | 1754 | 1902 |
| Подтверждение заказа | 1025 | 1099 |
| Транзакция | 623 | 642 |